## Object detection metrics and errors 
Object detection and instance segmentation primarily use one metric to judge performance: `mean Average Precision (mAP )`. How do we interpret it? For a metric like `accuracy` we know that 90% means that the model is wrong `10%` of the time. we will see the following things in this blog.
- what does a mAP of `63.2%` actually mean? 
- why do we use `mAP` as a metric in object detection? 
- what kind of errors are associated with your model which is hampering your overall score?
    - A `false positive` can be a 
        - duplicate detection
        - misclassification
        - mislocalization
        - confusion with background
        - both a misclassification and mislocalization.
    - a false negative could be a 
        - completely missed ground truth, 
        - potentially correct prediction could have just been misclassified or mislocalized. 

There are 3 places our detector can affect mAP 
- outputting false positives during the matching step, 
- not outputting true positives (i.e., false negatives) for computing recall, 
- having incorrect calibration (i.e., outputting a higher confidence for a false positive then a true positive).

The difficulty of the code has increased because
- we have multiple classes 
- we have multiple thresholds 
- we have to do for both mask and bbox
- we have multiple metrics [small, medium, large, mAR, mAP]


we will simplify the code by 
- using one single threshold (mAP 50)
- only bbox 
- only mAP 

First lets calculate `mAP` of the model. we will First use `torchvision` retinanet on COCO dataset

In [36]:
import numpy as np 
import pandas as pd
import torchvision
import torch
from PIL import Image
from pathlib import Path
import pandas as pd
from pycocotools.coco import COCO
from torchvision.ops.boxes import _box_xywh_to_xyxy as xywh_to_xyxy
from typing import Union

> Load the model.

In [2]:
weights = torchvision.models.detection.retinanet.RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1
model = torchvision.models.detection.retinanet_resnet50_fpn_v2(weights = weights, num_classes=91)
model.eval()
preprocess= weights.DEFAULT.transforms()

> Load the data

In [6]:
class COCODataset:
    def __init__(self, img_root, annot_loc, transforms=None):
        self.coco = COCO(annot_loc)
        self.img_root = Path(img_root)
        self.img_ids = list(self.coco.imgs.keys())
        self.transforms = transforms
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_meta = self.coco.imgs[img_id]
        img_loc = self.img_root / img_meta["file_name"]
        img = Image.open(img_loc)
        
        ## Load annotations 
        annot_ids = self.coco.getAnnIds(imgIds=[img_id])
        annots = self.coco.loadAnns(ids = annot_ids)
        
        if self.transforms is not None:
            return self.transforms(img).unsqueeze(0), annots
        return img, annots
    
    def __len__(self):
        return len(self.coco.imgs) 

In [7]:
annot_loc = "data/coco/annotations/instances_val2017.json"
img_root = "data/coco/val2017/"
ds = COCODataset(img_root, annot_loc=annot_loc, transforms=preprocess)

loading annotations into memory...
Done (t=0.59s)
creating index...
index created!


> Load a single image with its annotations.

In [8]:
img, annots = ds[10]

> Generate predictions

- This model uses an nms threshold of 0.5. check `model.nms_thresh`
- The output is a dict with boxes, scores, labels as keys. 

In [9]:
with torch.no_grad():
    output = model(img)
output[0].keys()

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


## check FP's, FN's and TP's for each bbox.
Each bbox (both gt and pred), we need to assign TP, FP or FN label. We will do this at 3 levels
- single image single class `SISCE` 
- single image all classes `SIE`
- Dataset level `DE`
where `E` stands for Error. 

### SISCE: single image, single class error
- this works on single image and for a single class.
- It calculates IOU between gt and pred of the same class bboxes. 
- For gt, it assigns TP or FN label
- For preds, it assigns TP or FP label.
- creats gt and preds df with scores.

In [11]:
pred_boxes = output[0]["boxes"]
pred_scores = output[0]["scores"]
pred_labels = output[0]["labels"]
print(pred_boxes.shape, pred_scores.shape, pred_labels.shape)

torch.Size([182, 4]) torch.Size([182]) torch.Size([182])


In [12]:
gt_boxes = torch.Tensor([i["bbox"] for i in annots])
gt_boxes = xywh_to_xyxy(gt_boxes)
gt_labels = torch.Tensor([i["category_id"] for i in annots])
print(gt_labels.shape, gt_boxes.shape)

torch.Size([26]) torch.Size([26, 4])


for each predicted bounding box we need to see if it has a correct match or not. we will do this for each class in each image first. 

In [13]:
categories = list(ds.coco.cats.keys())
req_cat = categories[0]
req_gt_boxes = gt_boxes[gt_labels ==1]
req_pred_boxes = pred_boxes[pred_labels == 1]
req_pred_scores = pred_scores[pred_labels==1]
print(req_cat, req_pred_boxes.shape, req_pred_scores.shape, req_gt_boxes.shape)

1 torch.Size([57, 4]) torch.Size([57]) torch.Size([13, 4])


In [122]:
class SISCE:
    #SingleImage_SingleClassError
    #
    def __init__(self, iou_thr):
        self.iou_thr = iou_thr
    
    def find_all_matches(self, c1: Union[torch.Tensor, np.asarray], c2:Union[torch.Tensor, np.asarray]):
        all_matches = np.empty((0, 3))
        if (len(c1) == 0) or (len(c2) == 0):
            return all_matches
        iou = torchvision.ops.box_iou(c1, c2).numpy()
        want_idx = np.where(iou > self.iou_thr)
        all_matches = []
        for i in range(want_idx[0].shape[0]):
            all_matches.append(
                [
                    want_idx[0][i],
                    want_idx[1][i],
                    iou[want_idx[0][i], want_idx[1][i]],
                ]
            )
        ## [N, 3]: i, j, score. [gt_index, pred_index, iou_score]
        all_matches = np.array(all_matches)
        if all_matches.shape[0] > 0:  # if there is match
            all_matches = all_matches[all_matches[:, 2].argsort()[::-1]]
            all_matches = all_matches[np.unique(all_matches[:, 1], return_index=True)[1]]
            all_matches = all_matches[all_matches[:, 2].argsort()[::-1]]
            all_matches = all_matches[np.unique(all_matches[:, 0], return_index=True)[1]]
        return all_matches 
    
    def find_gt_metrics(self, c1: Union[torch.Tensor, np.asarray], all_matches: np.asarray):
        # wheather a gt bbox is TP(matcheed with a predication) or FN(was not recognized by the alogrithm)
        c1_tt = []
        for num, gt_ctscanannot in enumerate(c1):
            TP_FP_FN = "FN"
            if len(all_matches) > 0:
                if num in all_matches[:, 0].tolist():
                    TP_FP_FN = "TP"
            c1_tt.append(TP_FP_FN)
        assert len(c1) == len(c1_tt), "there is a mismatch between number of nodules and labels"
        return c1_tt
    
    def get_gt_pd(self, c1: Union[torch.Tensor, np.asarray], all_matches: np.asarray):
        c1_tt = self.find_gt_metrics(c1, all_matches)
        return self.to_df(c1, c1_tt)
    
    def to_df(self, c1, c1_tt):
        if isinstance(c1, torch.Tensor):
            c1 = c1.numpy()
        df = pd.DataFrame(c1)
        df.columns = ["x1", "y1", "x2", "y2"]
        df["TP_FP_FN"] = c1_tt
        return df 
    
    
    def get_pred_pd(self, c1: Union[torch.Tensor, np.asarray], scores: Union[torch.Tensor, np.asarray], all_matches: np.asarray):
        c2_tt = self.find_pred_metrics(c1, all_matches)
        df = self.to_df(c1, [i[0] for i in c2_tt])
        df["iou_score"] = [i[1] for i in c2_tt]
        if isinstance(scores, torch.Tensor):
            scores = scores.numpy()
        df["cnf_score"] = scores
        return df 
        
    
    def find_pred_metrics(self, c1: Union[torch.Tensor, np.asarray], all_matches: np.asarray):
        c2_tt = []
        for num, pred_ctscaannot in enumerate(c1):
            iou_score, TP_FP_FN = 0.0, "FP"
            iou_score = 0.0
            if len(all_matches) > 0:
                if num in np.int64(all_matches[:, 1]):
                    index = int(np.where(np.int64(all_matches[:, 1]) == num)[0])
                    iou_score = all_matches[index][2]
                    TP_FP_FN = "TP"
            c2_tt.append([TP_FP_FN, iou_score])
        assert len(c1) == len(c2_tt), "there is a mismatch between number of nodules and labels"
        return c2_tt
    
    def forward(self, gt, pred, scores):
        matches = self.find_all_matches(gt, pred)
        preds_df = self.get_pred_pd(pred, scores, matches)
        gt_df = self.get_gt_pd(gt, matches)
        return gt_df, preds_df 

In [123]:
error = SISCE(0.5)

In [124]:
all_matches = error.find_all_matches(req_gt_boxes, req_pred_boxes)
all_matches

array([[ 0.        ,  1.        ,  0.76189595],
       [ 1.        ,  4.        ,  0.84801769],
       [ 2.        ,  0.        ,  0.92562395],
       [ 3.        ,  2.        ,  0.88024729],
       [ 4.        ,  8.        ,  0.63157892],
       [ 5.        ,  6.        ,  0.88249034],
       [ 6.        , 10.        ,  0.8462857 ],
       [ 7.        , 11.        ,  0.87101638],
       [ 8.        , 13.        ,  0.77039939],
       [ 9.        , 17.        ,  0.84931129],
       [10.        ,  7.        ,  0.89238977],
       [11.        ,  3.        ,  0.85208613],
       [12.        ,  5.        ,  0.65539688]])

In [125]:
pred_df_sisc  = error.get_pred_pd(req_pred_boxes, req_pred_scores, all_matches)
print(pred_df_sisc.shape)
pred_df_sisc.head()

(57, 7)


,x1,y1,x2,y2,TP_FP_FN,iou_score,cnf_score
0,0.210682,265.643738,110.534225,422.256592,TP,0.925624,0.875509
1,313.360962,284.719269,392.871094,421.843689,TP,0.761896,0.829786
2,429.111725,274.314087,533.618408,404.633026,TP,0.880247,0.819460
3,494.161133,276.724335,586.270264,385.799164,TP,0.852086,0.804641
4,273.743530,291.526611,330.614044,415.199249,TP,0.848018,0.783450


In [126]:
gt_df_sisc = error.get_gt_pd(req_gt_boxes, all_matches)
print(gt_df_sisc.shape)
gt_df_sisc.head()

(13, 5)


,x1,y1,x2,y2,TP_FP_FN
0,322.570007,290.809998,387.660004,418.429993,TP
1,273.640015,292.109985,324.590027,421.759979,TP
2,1.920000,266.880005,116.369995,422.669983,TP
3,424.119995,270.589996,531.589966,400.130005,TP
4,259.269989,281.000000,285.289978,316.100006,TP


### SIE: single image error 
- this basically evaluates at image level across all the classes.
- in case if you want to limit to single class we can use the earlier function or set the class label here. 

In [155]:
class SIE:
    # Single image error
    def __init__(self, iou_thr):
        self.sisce = SISCE(iou_thr=iou_thr)
    
    def forward_class(self, 
                      gts: torch.Tensor, 
                      gt_labels: torch.Tensor, 
                      preds: torch.Tensor, 
                      pred_scores: torch.Tensor, 
                      pred_labels: torch.Tensor, 
                      req_label: int):
        req_gt_boxes = gts[gt_labels ==req_label]
        req_pred_boxes = preds[pred_labels == req_label]
        req_pred_scores = pred_scores[pred_labels == req_label]
        gt_df, pred_df = self.sisce.forward(req_gt_boxes, req_pred_boxes, req_pred_scores)
        gt_df["label"] = gt_labels[gt_labels == req_label].numpy()
        pred_df["label"] = pred_labels[pred_labels==req_label]
        return gt_df, pred_df 
    
    def forward(self, 
                gt: torch.Tensor, 
                gt_labels: torch.Tensor, 
                pred: torch.Tensor, 
                pred_score: torch.Tensor, 
                pred_labels: torch.Tensor,):
        gts_df, preds_df = [], []
        labels = torch.hstack([pred_labels, gt_labels]).unique()
        for label in labels.unique():
            gt_df, pred_df = self.forward_class(gt, gt_labels, pred, pred_score, pred_labels, req_label= label)
            gts_df.append(gt_df)
            preds_df.append(pred_df)            
        return pd.concat(gts_df).reset_index(drop=True), pd.concat(preds_df).reset_index(drop=True)    

In [156]:
img_error = SIE(iou_thr=0.5)

In [159]:
gts_df, preds_df = img_error.forward(gt_boxes, gt_labels, pred_boxes, pred_scores, pred_labels)
print(gts_df.shape, preds_df.shape)

(26, 6) (182, 8)


In [160]:
gts_df.head()

,x1,y1,x2,y2,TP_FP_FN,label
0,322.570007,290.809998,387.660004,418.429993,TP,1.0
1,273.640015,292.109985,324.590027,421.759979,TP,1.0
2,1.920000,266.880005,116.369995,422.669983,TP,1.0
3,424.119995,270.589996,531.589966,400.130005,TP,1.0
4,259.269989,281.000000,285.289978,316.100006,TP,1.0


In [161]:
preds_df.head()

,x1,y1,x2,y2,TP_FP_FN,iou_score,cnf_score,label
0,0.210682,265.643738,110.534225,422.256592,TP,0.925624,0.875509,1
1,313.360962,284.719269,392.871094,421.843689,TP,0.761896,0.829786,1
2,429.111725,274.314087,533.618408,404.633026,TP,0.880247,0.819460,1
3,494.161133,276.724335,586.270264,385.799164,TP,0.852086,0.804641,1
4,273.743530,291.526611,330.614044,415.199249,TP,0.848018,0.783450,1
